In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import os

import datashader as ds
import colorcet


#, pandas as pd, colorcet
#df  = pd.read_csv('census.csv')
#cvs = ds.Canvas(plot_width=850, plot_height=500)
#agg = cvs.points(df, 'longitude', 'latitude')
#img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')



In [ ]:
from astropy import units as u
import astropy.coordinates as coord

In [ ]:
#infilenames = ['GaiaSource_000000-003111.csv.gz']
#infilenames += ['GaiaSource_668805-668856.csv.gz']
#infilenames += ['GaiaSource_472714-472767.csv.gz']
#infilenames += ['GaiaSource_786097-786431.csv.gz']


path = '/home/bellis/GAIA_data'
#path = '/home/bellis/GAIA/'



files = (filename for filename in os.listdir(path)
         if os.path.isfile(os.path.join(path,filename)) and filename.find('.h5')>=0)
    

infilenames = []
icount = 0
for f in files:
    infilenames.append(f"{path}/{f}")
    #print(f)
    icount += 1
    #if icount>=5:
    #    break
          
print(f"# files: {len(infilenames)}")

In [ ]:
li = []

nfiles = len(infilenames)

#for infilename in infilenames[0:5]:
for i,infilename in enumerate(infilenames):

    if i%100==0:
        print(i,nfiles,infilename)
        
    #print(infilename)
    df = None
    if infilename.find('.csv.gz')>=0:
        df = pd.read_csv(infilename, compression='gzip')
    elif infilename.find('.h5')>=0:
        df = pd.read_hdf(infilename)

    li.append(df)
    
df = pd.concat(li, axis=0, ignore_index=True)
    

Should also check out VisIt

https://visit-dav.github.io/visit-website/about/

And Paraview

https://www.paraview.org/


In [ ]:
#df = pd.read_hdf(infilenames[0])
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(df, 'ra', 'dec')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

In [ ]:
df['dr2_radial_velocity'].hist(bins=100)

print(len(df))

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(df, 'l', 'b')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

In [ ]:
print(min(df['l']), max(df['l']))
print(min(df['b']), max(df['b']))

In [ ]:
nvals = len(df)
print(nvals)

nsamples = 1000

x = np.arange(0,nvals,1).astype(int)
np.random.shuffle(x)

indices = x[0:nsamples]

#print(indices)

df_sample = df.iloc[indices]

name = f"gaia_subset_n{nsamples}.h5"

#df_sample.to_hdf(name,'df')

df_sample

plt.figure()
plt.plot(df_sample['ra'],df_sample['dec'],'.',markersize=1)

In [ ]:
#df = df[df['parallax'].notna()]

#galactic_latitude = df_sample['b'].values
#galactic_longitude = df_sample['l'].values

#parallax = df_sample['parallax'].values
#ra = df_sample['ra'].values
#dec = df_sample['dec'].values

parallax = df['parallax'].values
ra = df['ra'].values
dec = df['dec'].values


#print(parallax)

print(parallax[parallax!=parallax])

conversion = 0.001 # Convert milliarcsec to arcsec
r = 1/(parallax*conversion) # Distance in Mpc

mask = r>0

#print(r[mask][r[mask]==r[mask]])

#for i in galactic_longitude:
#    if isinstance(i, float)==True:
#        base=1
#    else:
#        print(i)
#distance = [1/(n*0.001) for n in parallax]

print()


#c= SkyCoord(galactic_longitude[mask]*u.deg,galactic_latitude[mask]*u.deg,distance=r*u.Mpc,frame='galactic')

c1 = coord.SkyCoord(ra=ra[mask]*u.degree, dec=dec[mask]*u.degree,
                    distance=(parallax[mask]*u.mas).to(u.pc, u.parallax()),
                    #pm_ra_cosdec=372.72*u.mas/u.yr,
                    #pm_dec=-483.69*u.mas/u.yr,
                    #radial_velocity=0.37*u.km/u.s,
                    frame='icrs')

gc1 = c1.transform_to(coord.Galactocentric)

v_sun = [11.1, 244, 7.25] * (u.km / u.s)  # [vx, vy, vz]
gc_frame = coord.Galactocentric(
    galcen_distance=8*u.kpc,
    galcen_v_sun=v_sun,
    z_sun=0*u.pc)

gc2 = c1.transform_to(gc_frame)

#gc_frame= Galactocentric(galcen_distance=8.1*u.kpc,
#                                   z_sun=25.*u.pc)
#gc = c.transform_to(gc_frame)
#gc.representation_type = 'cylindrical'

In [ ]:
dfgc = pd.DataFrame()

mask = (gc2.x>-10000*u.pc) & (gc2.x<10000*u.pc) & \
     (gc2.y>-10000*u.pc) & (gc2.y<10000*u.pc) & \
     (gc2.z>-10000*u.pc) & (gc2.z<10000*u.pc)
    

dfgc['x'] = gc2.x[mask]
dfgc['y'] = gc2.y[mask]
dfgc['z'] = gc2.z[mask]

In [ ]:
gc2.x

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(dfgc, 'x', 'y')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(dfgc, 'x', 'z')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

In [ ]:
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(dfgc, 'y', 'z')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

In [ ]:
len(dfgc)

In [ ]:
#plt.figure()
#plt.plot(gc2.x,gc2.y,'.',markersize=1)
#plt.xlim(-10000,10000)
#plt.ylim(-10000,10000)

## Uncertainties

In [ ]:
plt.figure(figsize=(16,4))
rv = df['dr2_radial_velocity']
rverr = df['dr2_radial_velocity_error']

plt.subplot(1,3,1)
plt.hist(rv,bins=100, range=(-200,200))
plt.xlabel('Radial velocity')

plt.subplot(1,3,2)
plt.hist(rverr,bins=100,range=(0,10))
plt.xlabel('Radial velocity error')

plt.subplot(1,3,3)
plt.hist(rverr/rv,bins=100,range=(0,2))
plt.xlabel('Radial velocity error / radial velocity')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(1,3,1)
plt.hist(r,bins=100,range=(0,2e4))
plt.xlabel('Distance (Mpc)')

plt.subplot(1,3,2)
plt.plot(r,rv,'.',markersize=0.5,alpha=0.05)
plt.xlabel('Distance (Mpc)')
plt.xlim(0,2e4)
plt.ylim(-200,200)


plt.subplot(1,3,3)
plt.plot(r,rverr,'.',markersize=0.5,alpha=0.05)
plt.xlabel('Distance (Mpc)')
plt.xlim(0,2e4)
plt.ylim(0,10)


In [ ]:
plt.figure(figsize=(16,4))

mask = (r>0) & (r<2e5)

plt.subplot(1,3,1)
plt.hist(r,bins=100,range=(0,2e4))
plt.xlabel('Distance (Mpc)')

plt.subplot(1,3,2)
plt.hist2d(r[mask],rv[mask],bins=(100,100),range=[[0,2e4],[-200,200]],cmap=plt.cm.jet)
plt.xlabel('Distance (Mpc)')
plt.xlim(0,2e4)
plt.ylim(-200,200)

plt.subplot(1,3,3)
plt.hist2d(r[mask],rverr[mask],bins=(100,100),range=[[0,2e4],[0,2]],cmap=plt.cm.jet)
plt.xlabel('Distance (Mpc)')
plt.xlim(0,2e4)

conversion = 0.001 # Convert milliarcsec to arcsec
r = 1/(parallax*conversion) # Distance in Mpc



## Velocities

Reference for units and data dictionary

https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html

In [ ]:
#'''
ra = df['ra'].values
dec = df['dec'].values
parallax = df['parallax'].values
pm_ra = df['pmra'].values
pm_dec = df['pmdec'].values
rv = df['dr2_radial_velocity'].values
#'''


'''
ra = df_sample['ra'].values
dec = df_sample['dec'].values
parallax = df_sample['parallax'].values
pm_ra = df_sample['pmra'].values
pm_dec = df_sample['pmdec'].values
rv = df_sample['dr2_radial_velocity'].values
'''

# Is this right?
pm_ra_cosdec = pm_ra*np.cos(pm_dec)

conversion = 0.001 # Convert milliarcsec to arcsec
r = 1/(parallax*conversion) # Distance in Mpc

mask = (r>0) & (r<2e5) # Cut on distance
#mask = (r>0) & (r<1000) # Cut on distance


c1_with_vel = coord.SkyCoord(ra=ra[mask]*u.degree, dec=dec[mask]*u.degree,
                    distance=(parallax[mask]*u.mas).to(u.pc, u.parallax()),
                    pm_ra_cosdec=pm_ra_cosdec[mask]*u.mas/u.yr,
                    pm_dec=pm_dec[mask]*u.mas/u.yr,
                    radial_velocity=rv[mask]*u.km/u.s,
                    frame='icrs')

v_sun = [11.1, 244, 7.25] * (u.km / u.s)  # [vx, vy, vz]
gc_frame = coord.Galactocentric(
    galcen_distance=8*u.kpc,
    galcen_v_sun=v_sun,
    z_sun=0*u.pc)

gc_with_vel = c1_with_vel.transform_to(gc_frame)


In [ ]:
vx = gc_with_vel.velocity.d_x
vy = gc_with_vel.velocity.d_y
vz = gc_with_vel.velocity.d_z

In [ ]:
plt.plot(vx,vy,'.',markersize=0.5,alpha=0.9)
plt.xlim(-200,200)
plt.ylim(-200,200)

In [ ]:
#vx.value

In [ ]:
plt.figure(figsize=(12,12))
plt.hist2d(vx.value,vy.value,bins=(100,100),range=[[-200,200],[0,400]],cmap=plt.cm.jet);


# Velocity

In [ ]:
x = gc_with_vel.cartesian.x
y = gc_with_vel.cartesian.y
z = gc_with_vel.cartesian.z

vx = gc_with_vel.velocity.d_x
vy = gc_with_vel.velocity.d_y
vz = gc_with_vel.velocity.d_z

r = np.sqrt(x*x + y*y + z*z)

vmag = np.sqrt(vx*vx + vy*vy + vz*vz)

In [ ]:
plt.figure(figsize=(15,9))
plt.subplot(2,2,1)
plt.hist(z.value,bins=100,range=(-1500,1500));

plt.subplot(2,2,2)
plt.hist(r.value,bins=100,range=(0,15000));

plt.subplot(2,2,3)
plt.hist(vmag.value,bins=100,range=(0,500));

In [ ]:
yvals = []
yerr = []
xvals = []
nvals = []

rwidth = 200

for i in np.arange(0,14000,rwidth):
    mask = (r.value>i) & (r.value<(i+rwidth)) & (np.abs(z.value)<200)
    print(i,i+rwidth,len(r[mask]))

    yvals.append(np.mean(vmag.value[mask]))
    yerr.append(np.std(vmag.value[mask]))
    xvals.append((i + (i+rwidth))/2.0)
    nvals.append(len(r[mask]))

yvals = np.array(yvals)
xvals = np.array(xvals)
yerr = np.array(yerr)
nvals = np.array(nvals)
    


In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(2,2,1)
plt.errorbar(xvals,yvals,yerr=yerr,fmt='o',markersize=5)
plt.ylim(0,300)

plt.subplot(2,2,2)
plt.errorbar(xvals,nvals,yerr=1)
#plt.ylim(0,300)

plt.subplot(2,2,3)
plt.errorbar(xvals,yvals,yerr=yerr/nvals,fmt='o',markersize=5)
plt.ylim(0,300)

plt.figure(figsize=(12,5))
plt.errorbar(xvals,yvals,yerr=yerr,fmt='o',markersize=5)
plt.xlabel(r'r (Mpc)',fontsize=24)
plt.ylabel(r'v (km/s)',fontsize=24)
plt.ylim(0,300)


plt.figure(figsize=(12,5))
plt.errorbar(xvals,yvals,yerr=yerr/nvals,fmt='o',markersize=5)
plt.xlabel(r'r (Mpc)',fontsize=24)
plt.ylabel(r'v (km/s)',fontsize=24)
plt.ylim(0,300)


In [ ]:


plt.figure(figsize=(12,5))
plt.errorbar(xvals,yvals,yerr=yerr,fmt='o',markersize=5)
plt.xlabel(r'r (Mpc)',fontsize=24)
plt.ylabel(r'v (km/s)',fontsize=24)
plt.ylim(0,300)


plt.figure(figsize=(12,5))
plt.errorbar(xvals,yvals,yerr=yerr/nvals,fmt='o',markersize=5)
plt.xlabel(r'r (Mpc)',fontsize=24)
plt.ylabel(r'v (km/s)',fontsize=24)
plt.ylim(0,300)

In [ ]:
#plt.plot(r,vmag,'.',alpha=0.1)

In [ ]:
len(r)

In [ ]:
# Animation
from matplotlib.animation import FuncAnimation
from matplotlib import rc


In [ ]:
#'''
ra = df_sample['ra'].values
dec = df_sample['dec'].values
parallax = df_sample['parallax'].values
pm_ra = df_sample['pmra'].values
pm_dec = df_sample['pmdec'].values
rv = df_sample['dr2_radial_velocity'].values
#'''

# Is this right?
pm_ra_cosdec = pm_ra*np.cos(pm_dec)

conversion = 0.001 # Convert milliarcsec to arcsec
r = 1/(parallax*conversion) # Distance in Mpc

mask = (r>0) & (r<2e5) # Cut on distance
#mask = (r>0) & (r<1000) # Cut on distance


gc_with_vel_anim = coord.SkyCoord(ra=ra[mask]*u.degree, dec=dec[mask]*u.degree,
                    distance=(parallax[mask]*u.mas).to(u.pc, u.parallax()),
                    pm_ra_cosdec=pm_ra_cosdec[mask]*u.mas/u.yr,
                    pm_dec=pm_dec[mask]*u.mas/u.yr,
                    radial_velocity=rv[mask]*u.km/u.s,
                    frame='icrs')

v_sun = [11.1, 244, 7.25] * (u.km / u.s)  # [vx, vy, vz]
gc_frame = coord.Galactocentric(
    galcen_distance=8*u.kpc,
    galcen_v_sun=v_sun,
    z_sun=0*u.pc)

gc_with_vel_anim = gc_with_vel_anim.transform_to(gc_frame)


In [ ]:
x = gc_with_vel_anim.x.value
y = gc_with_vel_anim.y.value
z = gc_with_vel_anim.z.value

vx = gc_with_vel_anim.velocity.d_x.value
vy = gc_with_vel_anim.velocity.d_y.value
vz = gc_with_vel_anim.velocity.d_z.value

print(len(x), len(vx))
print(len(y), len(vy))

In [ ]:
fig = plt.figure(figsize=(12,8))   #plotting
ax = plt.axes(xlim=(min(x),max(x)), ylim=(min(y),max(y)))
#ax = plt.axes(xlim=(-10000,-5000), ylim=(-5000,5000))


time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

#plt.xlabel("Right Ascension")
#plt.ylabel("Declination")
#plt.title("RA vs DEC" )


#####################################################################
point = ax.scatter([], [],s=5,alpha=0.2)

#### Another approach with scatter plot
# https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot

def init():
    time_text.set_text('')
    return point, time_text,

def animate(i):
    
    if i%100==0:
        print(i)
    #cmap = cm.Blues
    dt = 0.05
    t = i*dt
    
    x1 = x + (t*vx)
    y1 = y + (t*vy)
    
    xy = np.c_[x1, y1]
    point.set_offsets(xy)
    #point.set_sizes(10*1/(distance/1000))

    #time_text.set_text(str(dt*i)+" years")

    return point, time_text,

plt.close()

In [ ]:
    
nframes=500

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=nframes, interval=25, blit=True,  save_count=nframes)


In [ ]:
rc('animation', html='jshtml')
anim

# Write out small sample



In [ ]:
len(df_sample)

[GAIA source documentation](https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html)

In [ ]:
df.columns

In [ ]:
np.unique(df['dr2_radial_velocity'])

In [ ]:
rv = df['dr2_radial_velocity']
rv = rv.dropna()

print(f"{len(rv)} entries with rv as a number")
print(f"{len(df)} total entries")

In [ ]:
plt.hist(rv,bins=200);
plt.yscale('log')

In [ ]:
#dfgood = df[~df['dr2_radial_velocity'].isnull()]
dfgood = df

In [ ]:
len(dfgood)

dfgood['ra']

In [ ]:
x = dfgood['dr2_radial_velocity']
y = dfgood['phot_g_mean_mag']
y0 = df['phot_g_mean_mag']

plt.figure(figsize=(12,8))
plt.hist(y0,bins=50,range=(0,25))
plt.hist(y,bins=50,range=(0,25));

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x,y,'.',alpha=0.5,markersize=0.5)

https://en.wikipedia.org/wiki/Galactic_coordinate_system

https://astronomy.stackexchange.com/questions/35536/transforming-galactic-coordinates-in-cartesian-with-distance

https://en.wikipedia.org/wiki/Stellar_parallax

[Parsec](https://en.wikipedia.org/wiki/Parsec) is the units for $1/\textrm{parallax}$. 1 parsec = 3.26 light-years.

I think the units of parsecs is milliarcsecs (mas).

What to do about negative parallax?

https://astronomy.stackexchange.com/questions/26250/what-is-the-proper-interpretation-of-a-negative-parallax

Also, ecliptic coordinates are in the data

https://en.wikipedia.org/wiki/Ecliptic_coordinate_system

and l and b are in there explicitly. 

In [ ]:
c_icrs = SkyCoord(ra=10.68458*u.degree, dec=41.26917*u.degree, frame='icrs')
c_icrs.galactic

In [ ]:
#def galactic_coordinates(ra, dec, parallax):
def galactic_coordinates(l, b, parallax):

    #c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    
    #l = c.galactic.l.rad
    #b = c.galactic.b.rad
    
    lrad = np.deg2rad(l)
    brad = np.deg2rad(b)
    
    # Assume that parallax is passed in as milliarcsec
    conversion = 0.001 # Convert milliarcsec to arcsec
    one_over_p = 1/(parallax*conversion)

    x = one_over_p*np.cos(brad)*np.cos(lrad)
    y = one_over_p*np.cos(brad)*np.sin(lrad)
    z = one_over_p*np.sin(brad)
    
    return x,y,z
    

In [ ]:
ra_org = dfgood['ra']
dec_org = dfgood['dec']
p_org = dfgood['parallax']

l_org = dfgood['l']
b_org = dfgood['b']


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(ra_org,bins=50)

plt.subplot(1,3,2)
plt.hist(dec_org,bins=50)

plt.subplot(1,3,3)
plt.hist(p_org,bins=50)

plt.figure(figsize=(12,12))
plt.plot(ra_org,dec_org,'.',markersize=0.5,alpha=0.2)

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.hist(p_org,bins=50)


plt.subplot(2,2,2)
plt.hist(p_org[(p_org<50) & (p_org>0)],bins=50)

plt.subplot(2,2,3)
plt.hist(p_org[p_org>50],bins=50)

plt.subplot(2,2,4)
plt.hist(p_org[p_org<0.1],bins=50)

plt.figure()
plt.hist(1/p_org,bins=100)

1

In [ ]:
x,y,z = galactic_coordinates(10,20,0.5)

print(x,y,z)

Someone else's visualization of within 3000 parsecs

https://www.cosmos.esa.int/web/gaia/iow_20180614

In [ ]:
#mask = (p_org>0.1) & (ra_org == ra_org)
mask = (p_org>0.1)


#ra = ra_org[mask].values
#dec = dec_org[mask].values

l = l_org[mask].values
b = b_org[mask].values

p = p_org[mask].values
            

#x,y,z = galactic_coordinates(ra, dec, p)
x,y,z = galactic_coordinates(l, b, p)


#fig = plt.figure(figsize=(8,8))
#ax = fig.add_subplot(projection='3d')
#
#plt.plot(x,y,z,'.',markersize=0.5,alpha=0.2)

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(x,bins=50)

plt.subplot(1,3,2)
plt.hist(y,bins=50)

plt.subplot(1,3,3)
plt.hist(z,bins=50)

1

In [ ]:
rv = dfgood['dr2_radial_velocity']

In [ ]:
len(rv)

In [ ]:
plt.figure()
plt.hist(rv,bins=100)
plt.yscale('log')

1

In [ ]:
x,y,z

In [ ]:
rv

In [ ]:
df.columns

In [ ]:
df['ecl_lon']

In [ ]:
dfgood['parallax']

In [ ]:
#!pip install plotly

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
#px.scatter_3d(x,y,z,'.')
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(size=1))])

fig.show()

Proper motion

https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html
    

https://en.wikipedia.org/wiki/Proper_motion#:~:text=Proper%20motion%20is%20the%20astrometric,of%20the%20more%20distant%20stars.
    
 
pmra,pmdec
    

In [ ]:
pmra = dfgood['pmra']
pmdec = dfgood['pmdec']

In [ ]:
#mask = pmdec<1600
mask = pmdec<1000


#plt.plot(pmra[mask],pmdec[mask],'.')

In [ ]:
p = dfgood['parallax']
pm = dfgood['pm']


conversion = 0.001 # Convert milliarcsec to arcsec
one_over_p = 1/(p*conversion)
distance = one_over_p

vmag = np.sqrt(rv[mask]**2 + (distance[mask]*(pm[mask]/3.1e8))**2)

In [ ]:
plt.hist(vmag,bins=100,range=(0,1000));

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)

plt.hist(rv[mask],bins=100,range=(0,1000));

plt.subplot(1,2,2)

plt.hist(distance[mask]*pm[mask]/3.1e8,bins=100,range=(0,0.001))

1;

In [ ]:
plt.hist(pm[mask]/3.1e8,bins=100);

In [ ]:
max(distance[mask]*pm[mask]/3.1e8)

In [ ]:
nvals = len(df)
print(nvals)

nsamples = 100000

x = np.arange(0,nvals,1).astype(int)
np.random.shuffle(x)

indices = x[0:nsamples]

#print(indices)

df_sample = df.iloc[indices]

name = f"gaia_subset_n{nsamples}.h5"

df_sample.to_hdf(name,'df')

df_sample

p = df_sample['parallax']

#mask = (p_org>0.1) & (ra_org == ra_org)
mask = (p>0.1)


#ra = ra_org[mask].values
#dec = dec_org[mask].values

l = df_sample[mask]['l'].values
b = df_sample[mask]['b'].values

p = p[mask].values
            

#x,y,z = galactic_coordinates(ra, dec, p)
x,y,z = galactic_coordinates(l, b, p)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(size=1))])

fig.show()

# Animation

In [ ]:
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib import rc
from matplotlib import cm, colors

import matplotlib.animation as animation


In [ ]:
len(df)

In [ ]:
nsamples = 1000

x = np.arange(0,nvals,1).astype(int)
np.random.shuffle(x)

indices = x[0:nsamples]

#print(indices)

df_sample = df.iloc[indices]


In [ ]:
fig = plt.figure(figsize=(12,8))   #plotting
ax = plt.axes(xlim=(-0,360), ylim=(-100,100))

time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

plt.xlabel("Right Ascension")
plt.ylabel("Declination")
plt.title("RA vs DEC" )


ra = df_sample['ra'].values
dec = df_sample['dec'].values

pmra = df_sample['pmra'].values
pmdec = df_sample['pmdec'].values

print(len(distance))
print(len(ra))

parallax = df_sample['parallax'].values

P = parallax/1000
distance = 1/P    

pmmag2 = np.sqrt(pmra**2 + pmdec**2)

mask = pmmag<50

ra = ra[mask]
dec = dec[mask]

pmra = pmra[mask]
pmdec = pmdec[mask]   

distance = distance[mask]

print(len(ra), len(pmra))

#####################################################################
#point, = ax.plot([], [], 'k.', markersize=1)
point = ax.scatter([], [],s=10)

#### Another approach with scatter plot
# https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot

def init():
    time_text.set_text('')
    return point, time_text,

def animate(i):
    cmap = cm.Blues
    dt = 10000
    t = i*dt
    
    x1 = ra + (t*pmra/3.6e6)
    y1 = dec + (t*pmdec/3.6e6)
    
    xy = np.c_[x1, y1]
    point.set_offsets(xy)
    point.set_sizes(10*1/(distance/1000))

    time_text.set_text(str(dt*i)+" years")

    return point, time_text,

plt.close()

In [ ]:
    
npts=500

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=npts, interval=25, blit=True,  save_count=npts)


In [ ]:
rc('animation', html='jshtml')
anim

In [ ]:
anim.save('gaia_animation.mp4')


In [ ]:
ra_degrees = df['ra'].values
dec_degrees = df['dec'].values

pmra = df['pmra'].values
pmdec = df['pmdec'].values

pmmag = np.sqrt(pmra**2 + pmdec**2)

parallax = df['parallax'].values
P = parallax/1000
distance = 1/P    
    
plt.figure()
plt.hist(pmmag,bins=100,range=(0,50));

plt.figure()
plt.hist(distance,bins=100,range=(-1,10000));

In [ ]:
plt.plot(distance[pmmag<50],pmmag[pmmag<50],'.',markersize=1)
plt.xlim(0,10000)

In [ ]:
plt.figure()
plt.scatter([0,1],[0,1],marker='o',s=np.array([10,20]))

In [ ]:
? plt.scatter